# connect to playstation

In [9]:
from psnawp_api import PSNAWP
import os

psnawp = PSNAWP(os.getenv('psn'))

client = psnawp.me()


In [10]:
# This is you
client = psnawp.me()
print(client.get_profile_legacy())

print("hello word")

{'profile': {'onlineId': 'yupex', 'accountId': '3085305673771759439', 'npId': 'eXVwZXhAZDMuZnI=', 'avatarUrls': [{'size': 'l', 'avatarUrl': 'http://static-resource.np.community.playstation.net/avatar/3RD/UP40731301009_B479A733FF1EFD68AF30_L.png'}], 'plus': 1, 'aboutMe': '', 'languagesUsed': ['fr'], 'trophySummary': {'level': 210, 'progress': 43, 'earnedTrophies': {'platinum': 0, 'gold': 38, 'silver': 178, 'bronze': 725}}, 'isOfficiallyVerified': False, 'personalDetail': {'firstName': 'Francois', 'lastName': 'Lenne'}, 'personalDetailSharing': 'no', 'personalDetailSharingRequestMessageFlag': False, 'primaryOnlineStatus': 'offline', 'presences': [{'onlineStatus': 'offline', 'hasBroadcastData': False, 'lastOnlineDate': '2019-01-13T15:29:24Z'}], 'friendRelation': 'no', 'requestMessageFlag': False, 'blocking': False, 'following': False, 'consoleAvailability': {'availabilityStatus': 'offline'}}}
hello word


In [12]:
titles_with_stats = client.title_stats()

print(titles_with_stats)

In [13]:
import pandas as pd

# Supposons que TitleStatsListing a un attribut 'titles' qui est une liste d'objets TitleStats
data = [[ts.title_id, ts.name, ts.image_url, ts.category, ts.first_played_date_time, ts.last_played_date_time , ts.play_count, ts.play_duration] for ts in titles_with_stats]

# Convertir les données en un DataFrame
df = pd.DataFrame(data, columns=['title_id', 'title_name', 'image', 'category', 'first_played_date_time','last_played_date_time' ,'play_count', 'play_duration'])

print("hello word")

hello word
